# Train

In [2]:
import pasta_spaghettini_g3 as spa_g3
import pandas as pd
from tqdm import tqdm
import torch

PASTA = spa_g3.PASTA(input_size=30, hidden_size=8, num_layers=2, learning_rate=0.001)
PASTA.load_parameter('./parameters/param_spa_g3_ch')

data_route = '../../Dataset/League_of_Legends/features_tensor/'
match_list = pd.read_feather('./processed_ftr/match_result_train.ftr')
total_rows = match_list.shape[0]
# highest_acc = 0.992985 # time-reversed
highest_acc = 0.988595 # chronological
# highest_acc = 0.0 # time-reversed bce
# highest_acc = 0.0 # chronological bce

print("--Train start--")
for epoch in range(10):
    confusion_matrix, c_label = [0, 0, 0, 0], ["TN", "FP", "FN", "TP"]
    match_list = match_list.sample(frac=1).reset_index(drop=True)
    
    for idx in tqdm(range(total_rows), desc='[Epoch %1d] '%(epoch+1)):
        row = match_list.iloc[idx]
        match_id, win = row['match_no'], row['win']
        features = pd.read_pickle(f"{data_route}by_team_train_ch/{match_id}.pkl")

        winner, predict = PASTA.train(features, win)
        correct = 2*winner + predict
        confusion_matrix[correct] += 1

    tn, fp, fn, tp = confusion_matrix[0], confusion_matrix[1], confusion_matrix[2], confusion_matrix[3]
    acc = (tp+tn)/(tn+fp+fn+tp)
    pre = tp/(tp+fp) if (tp+fp) != 0 else 0
    rec = tp/(tp+fn) if (tp+fn) != 0 else 0
    f1 = 2 * (pre * rec) / (pre + rec)

    for idx in range(0, 4):
        print(f'{c_label[idx]}: {confusion_matrix[idx]} / ', end='')
    print('\n' + 'Accuracy: %.6f / Precision: %.6f / Recall: %.6f / f1 score: %.6f'%(acc, pre, rec, f1))

    if acc > highest_acc:
        print(f"New record. Saving parameters...\n")
        PASTA.save_parameter('./parameters/param_spa_g3_ch')
        highest_acc = acc
    else:
        print('Save temporary parameters...\n')
        PASTA.save_parameter('./parameters/tmp')
    
    torch.cuda.empty_cache()

--Train start--


[Epoch 1] : 100%|██████████| 200000/200000 [1:34:54<00:00, 35.12it/s]


TN: 96797 / FP: 1066 / FN: 1096 / TP: 101041 / 
Accuracy: 0.989190 / Precision: 0.989560 / Recall: 0.989269 / f1 score: 0.989415
New record. Saving parameters...



[Epoch 2] : 100%|██████████| 200000/200000 [1:34:31<00:00, 35.26it/s]


TN: 96841 / FP: 1022 / FN: 1065 / TP: 101072 / 
Accuracy: 0.989565 / Precision: 0.989990 / Recall: 0.989573 / f1 score: 0.989781
New record. Saving parameters...



[Epoch 3] : 100%|██████████| 200000/200000 [1:33:46<00:00, 35.55it/s]


TN: 96891 / FP: 972 / FN: 998 / TP: 101139 / 
Accuracy: 0.990150 / Precision: 0.990481 / Recall: 0.990229 / f1 score: 0.990355
New record. Saving parameters...



[Epoch 4] :  71%|███████   | 141203/200000 [1:05:56<28:28, 34.42it/s]

# Test

In [ ]:
import pasta_spaghettini_g3 as spa_g3
import pandas as pd
from tqdm import tqdm
import torch

PASTA = spa_g3.PASTA(input_size=30, hidden_size=8, num_layers=1, learning_rate=0.01)

data_route = '../../Dataset/League_of_Legends/features_tensor/'
match_list = pd.read_feather('./processed_ftr/match_result_test.ftr')
total_rows = match_list.shape[0]

print("--Test start--")
PASTA.load_parameter()
confusion_matrix, c_label = [0, 0, 0, 0], ["TN", "FP", "FN", "TP"]

for idx in tqdm(range(total_rows), desc='[Epoch %1d] '%(epoch+1)):
    row = match_list.iloc[idx]
    match_id, win = row['match_no'], row['win']
    features = pd.read_pickle(f"{data_route}by_team_train/{match_id}.pkl")

    winner, predict = PASTA.test(features, win)
    correct = 2*winner + predict
    confusion_matrix[correct] += 1

tn, fp, fn, tp = confusion_matrix[0], confusion_matrix[1], confusion_matrix[2], confusion_matrix[3]
acc = (tp+tn)/(tn+fp+fn+tp) * 100
pre = tp/(tp+fp) * 100 if (tp+fp) != 0 else 0
rec = tp/(tp+fn) * 100 if (tp+fn) != 0 else 0
f1 = 2 * (pre * rec) / (pre + rec)

for idx in range(0, 4):
    print(f'{c_label[idx]}: {confusion_matrix[idx]} / ', end='')
print('\n' + 'Accuracy: %4.2f / Precision: %.4f / Recall: %.4f / f1 score: %.4f'%(acc, pre, rec, f1))

torch.cuda.empty_cache()